# Load Data

In [87]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from aif360.algorithms.preprocessing import DisparateImpactRemover, LFR, Reweighing
from aif360.datasets import AdultDataset, CompasDataset, BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric

import pandas as pd
import numpy as np

seed = 64

In [2]:
ad_protected = 'sex'
ad_label = "income-per-year"
ad = AdultDataset(protected_attribute_names=[ad_protected],
    privileged_classes=[['Male']], categorical_features=[],
    features_to_keep=['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'])

ad_p = [{ad_protected: 1}]
ad_u = [{ad_protected: 0}]

comp_protected = 'race'
comp_label = 'two_year_recid'
comp = CompasDataset(protected_attribute_names=[comp_protected],
                     privileged_classes=[["Caucasian"]],
                     features_to_keep=["age_cat", "juv_fel_count", "juv_misd_count", "juv_other_count", "priors_count", "c_charge_degree", "c_charge_desc"])

comp_p = [{comp_protected: 1}]
comp_u = [{comp_protected: 0}]

# Fairness Functions

In [89]:
def model_accuracy(true, pred):
    print(f"ACCURACY: {accuracy_score(true, pred)}")
    print(f"RECALL: {recall_score(true, pred)}")
    print(f"PRECISION: {precision_score(true, pred)}")
    print(f"F1: {f1_score(true, pred)}")

In [3]:
def fairness_metrics(y, y_pred):
    """Calculate fairness for subgroup of population"""

    #Confusion Matrix
    cm=confusion_matrix(y, y_pred)
    TN, FP, FN, TP = cm.ravel()

    N = TP+FP+FN+TN #Total population
    ACC = (TP+TN)/N #Accuracy
    TPR = TP/(TP+FN) # True positive rate
    FPR = FP/(FP+TN) # False positive rate
    FNR = FN/(TP+FN) # False negative rate
    PPP = (TP + FP)/N # % predicted as positive

    return np.array([ACC, TPR, FPR, FNR, PPP])

In [4]:
def print_metrics(privileged_metrics, unprivileged_metrics):
  print("    |PRIVILEGED|UNPRIVILEGED")
  print(f" ACC|{round(privileged_metrics[0], 8)}|  {round(unprivileged_metrics[0], 8)}")
  print(f" TPR|{round(privileged_metrics[1], 8)}|  {round(unprivileged_metrics[1], 8)}")
  print(f" FPR|{round(privileged_metrics[2], 8)}|  {round(unprivileged_metrics[2], 8)}")
  print(f" FNR|{round(privileged_metrics[3], 8)}|  {round(unprivileged_metrics[3], 8)}")
  print(f" PPP|{round(privileged_metrics[4], 8)}|  {round(unprivileged_metrics[4], 8)}")

In [5]:
def print_results(dataset_name, priv_metrics, unpriv_metrics, binm):
    eo_tpr = priv_metrics[1] - unpriv_metrics[1]
    eo_fpr = priv_metrics[2] - unpriv_metrics[2]

    print("="*20)
    print(dataset_name)
    print("="*20)
    print(f"EQUALIZED ODDS - TPR: {eo_tpr}")
    print(f"EQUALIZED ODDS - FPR: {eo_fpr}")
    print(f"STAT PARITY DIFF: {binm.statistical_parity_difference()}")
    print(f"DISPARATE IMPACT: {binm.disparate_impact()}")
    print("="*20)

In [6]:
def fairness_cv(model, fair_algo, X, y, feature_names, prot_attr, label, prot_index, model_params=None, cv=5):
    if X.shape[0] != y.shape[0]:
        raise IndexError(f"Lengths of X and y do not match. X: {X.shape}, y: {y.shape}")
    
    X_red = np.delete(X, list(range(len(X)%cv)), axis=0)
    y_red = np.delete(y, list(range(len(X)%cv)), axis=0)

    X_split = np.split(X_red, cv, axis=0)
    y_split = np.split(y_red, cv, axis=0)

    data_split_df = []
    for section in range(len(X_split)):
        X_df = pd.DataFrame(data=X_split[section], columns=feature_names)
        y_df = pd.DataFrame(data=y_split[section], columns=[label])
        df = pd.concat([X_df, y_df], axis="columns")
        data_split_df.append(df)

    scores = []

    for iter in range(len(data_split_df)):
        # X_tr = data_split_df[iter].filter(items=feature_names, axis="columns")
        # y_tr = data_split_df[iter][label]

        # X_te = X_split[iter].filter(items=feature_names, axis="columns")
        # y_te = y_split[iter][label]

        data_split_df_cp = data_split_df.copy()
        data_split_df_cp.pop(iter)
        df_tr = pd.concat(data_split_df_cp, axis=0)
        df_tr.reset_index()
        feature_names_no_prot = feature_names.copy()
        feature_names_no_prot.remove(prot_attr)
        X_tr = df_tr.filter(items=feature_names_no_prot, axis="columns")
        s_tr = df_tr[prot_attr]

        data_bin = BinaryLabelDataset(df=data_split_df[iter], label_names=[label], protected_attribute_names=[prot_attr])

        if isinstance(fair_algo, DisparateImpactRemover):
            # X_te_transf = fair_algo.fit_transform(X_te)
            # y_te_transf = fair_algo.fit_transform(y_te)
            data_transf = fair_algo.fit_transform(data_bin)
        else:
            # X_te_transf = fair_algo.transform(X_te)
            # y_te_transf = fair_algo.transform(y_te)
            data_transf = fair_algo.transform(data_bin)
        
        X_te_transf = np.delete(data_transf.features, prot_index, axis=1)
        s_te = data_bin.protected_attributes.ravel()

        model.fit(X_tr, s_tr)
        s_pred = model.predict(X_te_transf)
        score = accuracy_score(s_te, s_pred)
        scores.append(score)
    
    return np.array(scores)
        

# Preprocessing

In [7]:
scaler = MinMaxScaler(copy=False)

ad_scaled = ad.copy()
ad_scaled.features = scaler.fit_transform(ad_scaled.features)

comp_scaled = comp.copy()
comp_scaled.features = scaler.fit_transform(comp_scaled.features)

ad_prot_index = ad_scaled.feature_names.index(ad_protected) # gets the index of the ad_protected attribute
comp_prot_index = comp_scaled.feature_names.index(comp_protected)


# Fairness Correction

## No FC

### Adult Income

In [90]:
ad_base_train, ad_base_test = ad_scaled.split([0.66])
ad_base_df = ad_base_test.convert_to_dataframe()[0]
ad_base_train_X = np.delete(ad_base_train.features, ad_prot_index, axis=1)
ad_base_train_y = ad_base_train.labels.ravel()
ad_base_test_X = np.delete(ad_base_test.features, ad_prot_index, axis=1)
ad_base_test_y = ad_base_test.labels.ravel()

ad_base_lr = LogisticRegression(random_state=seed)
ad_base_lr.fit(ad_base_train_X, ad_base_train_y)
ad_base_lr_pred = ad_base_lr.predict(ad_base_test_X)
model_accuracy(ad_base_test_y, ad_base_lr_pred)

ad_base_df_priv = ad_base_df[ad_base_df[ad_protected] == ad_p[0][ad_protected]]
ad_base_df_unpriv = ad_base_df[ad_base_df[ad_protected] == ad_u[0][ad_protected]]

ad_base_df_priv_X = ad_base_df_priv.drop(labels=[ad_protected, ad_label], axis="columns")
ad_base_df_priv_y = ad_base_df_priv[ad_label]

ad_base_df_unpriv_X = ad_base_df_unpriv.drop(labels=[ad_protected, ad_label], axis="columns")
ad_base_df_unpriv_y = ad_base_df_unpriv[ad_label]

ad_base_priv_pred = ad_base_lr.predict(ad_base_df_priv_X)
ad_base_unpriv_pred = ad_base_lr.predict(ad_base_df_unpriv_X)

ad_base_priv_metrics = fairness_metrics(ad_base_df_priv_y, ad_base_priv_pred)
ad_base_unpriv_metrics = fairness_metrics(ad_base_df_unpriv_y, ad_base_unpriv_pred)

ACCURACY: 0.812669356295538
RECALL: 0.37384806973848067
PRECISION: 0.715443279313632
F1: 0.491084573858989


/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


### Compas

In [ ]:
comp_base_train, comp_base_test = comp_scaled.split([0.66])
comp_base_df = comp_base_test.convert_to_dataframe()[0]
comp_base_train_X = np.delete(comp_base_train.features, comp_prot_index, axis=1)
comp_base_train_y = comp_base_train.labels.ravel()
comp_base_test_X = np.delete(comp_base_test.features, comp_prot_index, axis=1)
comp_base_test_y = comp_base_test.labels.ravel()

comp_base_lr = LogisticRegression(random_state=seed)
comp_base_lr.fit(comp_base_train_X, comp_base_train_y)
comp_base_lr_pred = comp_base_lr.predict(comp_base_test_X)
model_accuracy(comp_base_test_y, comp_base_lr_pred)

comp_base_df_priv = comp_base_df[comp_base_df[comp_protected] == comp_p[0][comp_protected]]
comp_base_df_unpriv = comp_base_df[comp_base_df[comp_protected] == comp_u[0][comp_protected]]

comp_base_df_priv_X = comp_base_df_priv.drop(labels=[comp_protected, comp_label], axis="columns")
comp_base_df_priv_y = comp_base_df_priv[comp_label]

comp_base_df_unpriv_X = comp_base_df_unpriv.drop(labels=[comp_protected, comp_label], axis="columns")
comp_base_df_unpriv_y = comp_base_df_unpriv[comp_label]

comp_base_priv_pred = comp_base_lr.predict(comp_base_df_priv_X)
comp_base_unpriv_pred = comp_base_lr.predict(comp_base_df_unpriv_X)

comp_base_priprint(f"ACCURACY: {v_metrics = fairness_metrics(comp_base_df_priv_y, comp_base_priv_pred)
comp_base_unpriv_metrics = fairness_metrics(comp_base_df_unpriv_y, comp_base_unpriv_pred)

ACCURACY: 0.6685741535526943
RECALL: 0.526694045174538
PRECISION: 0.6867469879518072
F1: 0.5961650203370134


/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


## Disparate Impact Removal

### Adult Income

In [10]:
ad_dir = DisparateImpactRemover(repair_level=1.0)
ad_dir_class, ad_dir_recon = ad_scaled.split([0.5])

ad_dir_class_train, ad_dir_class_test = ad_dir_class.split([0.66])
ad_dir_train = ad_dir.fit_transform(ad_dir_class_train)
ad_dir_test = ad_dir.fit_transform(ad_dir_class_test)

ad_dir_recon_train, ad_dir_recon_test = ad_dir_recon.split([0.66])
ad_dir_recon_train_repr = ad_dir.fit_transform(ad_dir_recon_train)
ad_dir_recon_test_repr = ad_dir.fit_transform(ad_dir_recon_test)

In [11]:
ad_dir_X_tr = np.delete(ad_dir_train.features, ad_prot_index, axis=1)  # deletes ad_protected attribute from X
ad_dir_X_te = np.delete(ad_dir_test.features, ad_prot_index, axis=1)  # deletes ad_protected attribute from X
ad_dir_y_tr = ad_dir_train.labels.ravel()
ad_dir_y_te = ad_dir_test.labels.ravel()

In [92]:
ad_dir_lr = LogisticRegression(random_state=seed)
ad_dir_lr.fit(ad_dir_X_tr, ad_dir_y_tr)
ad_dir_lr_pred = ad_dir_lr.predict(ad_dir_X_te)
model_accuracy(ad_dir_y_te, ad_dir_lr_pred)

ACCURACY: 0.8033477842003853
RECALL: 0.3014112903225806
PRECISION: 0.7076923076923077
F1: 0.42276422764227645


In [13]:
ad_dir_df = ad_dir_test.convert_to_dataframe()[0]
ad_dir_df_priv = ad_dir_df[ad_dir_df[ad_protected] == ad_p[0][ad_protected]]
ad_dir_df_unpriv = ad_dir_df[ad_dir_df[ad_protected] == ad_u[0][ad_protected]]

ad_dir_df_priv_X = ad_dir_df_priv.drop(labels=[ad_protected, ad_label], axis="columns")
ad_dir_df_priv_y = ad_dir_df_priv[ad_label]

ad_dir_df_unpriv_X = ad_dir_df_unpriv.drop(labels=[ad_protected, ad_label], axis="columns")
ad_dir_df_unpriv_y = ad_dir_df_unpriv[ad_label]

ad_dir_priv_pred = ad_dir_lr.predict(ad_dir_df_priv_X)
ad_dir_unpriv_pred = ad_dir_lr.predict(ad_dir_df_unpriv_X)

ad_dir_priv_metrics = fairness_metrics(ad_dir_df_priv_y, ad_dir_priv_pred)
ad_dir_unpriv_metrics = fairness_metrics(ad_dir_df_unpriv_y, ad_dir_unpriv_pred)

/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


### Compas

In [14]:
comp_di = DisparateImpactRemover(repair_level=1.0)
# comp_di_transf = comp_di.fit_transform(comp_scaled)
comp_di_class, comp_di_recon = comp_scaled.split([0.5])
comp_di_class_transf = comp_di.fit_transform(comp_di_class)
comp_di_train, comp_di_test = comp_di_class_transf.split([0.66])

In [15]:
comp_di_X_tr = np.delete(comp_di_train.features, comp_prot_index, axis=1)  # deletes comp_protected attribute from X
comp_di_X_te = np.delete(comp_di_test.features, comp_prot_index, axis=1)   # deletes comp_protected attribute from X
comp_di_y_tr = comp_di_train.labels.ravel()
comp_di_y_te = comp_di_test.labels.ravel()

In [93]:
comp_di_lr = LogisticRegression(random_state=seed)
comp_di_lr.fit(comp_di_X_tr, comp_di_y_tr)
comp_di_lr_pred = comp_di_lr.predict(comp_di_X_te)
model_accuracy(comp_di_y_te, comp_di_lr_pred)

ACCURACY: 0.6234509056244042
RECALL: 0.4576612903225806
PRECISION: 0.6430594900849859
F1: 0.5347467608951708


In [17]:
comp_di_df = comp_di_test.convert_to_dataframe()[0]
comp_di_df_priv = comp_di_df[comp_di_df[comp_protected] == comp_p[0][comp_protected]]
comp_di_df_unpriv = comp_di_df[comp_di_df[comp_protected] == comp_u[0][comp_protected]]

comp_di_df_priv_X = comp_di_df_priv.drop(labels=[comp_protected, comp_label], axis="columns")
comp_di_df_priv_y = comp_di_df_priv[comp_label]

comp_di_df_unpriv_X = comp_di_df_unpriv.drop(labels=[comp_protected, comp_label], axis="columns")
comp_di_df_unpriv_y = comp_di_df_unpriv[comp_label]

comp_di_priv_pred = comp_di_lr.predict(comp_di_df_priv_X)
comp_di_unpriv_pred = comp_di_lr.predict(comp_di_df_unpriv_X)

comp_di_priv_metrics = fairness_metrics(comp_di_df_priv_y, comp_di_priv_pred)
comp_di_unpriv_metrics = fairness_metrics(comp_di_df_unpriv_y, comp_di_unpriv_pred)

/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


## Learning Fair Representations

### Adult Income

In [18]:
ad_lfr_classification, ad_lfr_recon = ad.split([0.5], shuffle=True)
ad_lfr_orig_train, ad_lfr_orig_test = ad_lfr_classification.split([0.66], shuffle=True)

In [19]:
ad_scale = StandardScaler()
ad_lfr_orig_train.features = ad_scale.fit_transform(ad_lfr_orig_train.features)
ad_lfr_orig_test.features = ad_scale.transform(ad_lfr_orig_test.features)

In [20]:
ad_privileged_groups = [{'sex': 1}]
ad_unprivileged_groups = [{'sex': 0}]
    
ad_lfr = LFR(unprivileged_groups=ad_unprivileged_groups,
         privileged_groups=ad_privileged_groups,
         k=10, Ax=0.1, Ay=1.0, Az=2.0)
ad_lfr = ad_lfr.fit(ad_lfr_orig_train, maxiter=5000, maxfun=5000)

In [21]:
ad_lfr_transf_train = ad_lfr.transform(ad_lfr_orig_train)
ad_lfr_transf_test = ad_lfr.transform(ad_lfr_orig_test)
# ad_lfr_recon_transf_train = ad_lfr.transform(ad_lfr_recon_orig_train)
# ad_lfr_recon_transf_test = ad_lfr.transform(ad_lfr_recon_orig_test)

In [22]:
ad_lfr_X_tr = np.delete(ad_lfr_transf_train.features, ad_prot_index, axis=1)
ad_lfr_X_te = np.delete(ad_lfr_transf_test.features, ad_prot_index, axis=1)
ad_lfr_y_tr = ad_lfr_transf_train.labels.ravel()
ad_lfr_y_te = ad_lfr_transf_test.labels.ravel()

In [94]:
ad_lfr_lr = LogisticRegression(random_state=seed)
ad_lfr_lr.fit(ad_lfr_X_tr, ad_lfr_y_tr)
ad_lfr_lr_pred = ad_lfr_lr.predict(ad_lfr_X_te)
model_accuracy(ad_lfr_y_te, ad_lfr_lr_pred)

ACCURACY: 0.996266859344894
RECALL: 0.9788557213930348
PRECISION: 0.982521847690387
F1: 0.9806853582554517


In [24]:
ad_lfr_df = ad_lfr_transf_test.convert_to_dataframe()[0]
ad_lfr_df_priv = ad_lfr_df[ad_lfr_df[ad_protected] == ad_p[0][ad_protected]]
ad_lfr_df_unpriv = ad_lfr_df[ad_lfr_df[ad_protected] == ad_u[0][ad_protected]]

ad_lfr_df_priv_X = ad_lfr_df_priv.drop(labels=[ad_protected, ad_label], axis="columns")
ad_lfr_df_priv_y = ad_lfr_df_priv[ad_label]

ad_lfr_df_unpriv_X = ad_lfr_df_unpriv.drop(labels=[ad_protected, ad_label], axis="columns")
ad_lfr_df_unpriv_y = ad_lfr_df_unpriv[ad_label]

ad_lfr_priv_pred = ad_lfr_lr.predict(ad_lfr_df_priv_X)
ad_lfr_unpriv_pred = ad_lfr_lr.predict(ad_lfr_df_unpriv_X)

ad_lfr_priv_metrics = fairness_metrics(ad_lfr_df_priv_y, ad_lfr_priv_pred)
ad_lfr_unpriv_metrics = fairness_metrics(ad_lfr_df_unpriv_y, ad_lfr_unpriv_pred)

/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


### Compas

In [25]:
comp_df, metadata = comp.convert_to_dataframe()
comp_df = comp_df.filter(items=["race", "juv_fel_count", "juv_misd_count", "juv_other_count", "priors_count", "age_cat=Greater than 45", "age_cat=Less than 25", "two_year_recid", "c_charge_degree=F", "c_charge_degree=M"], axis="columns")
comp = BinaryLabelDataset(df=comp_df, label_names=metadata["label_names"], protected_attribute_names=metadata["protected_attribute_names"])

In [26]:
comp_lfr_classification, comp_lfr_recon = comp.split([0.5], shuffle=True)
comp_lfr_orig_train, comp_lfr_orig_test = comp_lfr_classification.split([0.66], shuffle=True)
# comp_lfr_recon_orig_train, comp_lfr_recon_orig_test = comp_recon.split([0.66], shuffle=True)

In [27]:
comp_scale = StandardScaler()
comp_lfr_orig_train.features = comp_scale.fit_transform(comp_lfr_orig_train.features)
comp_lfr_orig_test.features = comp_scale.fit_transform(comp_lfr_orig_test.features)

In [28]:
comp_privileged_groups = [{comp_protected: 1}]
comp_unprivileged_groups = [{comp_protected: 0}]

comp_lfr = LFR(unprivileged_groups=comp_unprivileged_groups,
               privileged_groups=comp_privileged_groups,
               k=10, Ax=0.005, Ay=0.75, Az=0.5)
comp_lfr = comp_lfr.fit(comp_lfr_orig_train, maxiter=5000, maxfun=5000)

In [29]:
comp_lfr_transf_train = comp_lfr.transform(comp_lfr_orig_train)
comp_lfr_transf_test = comp_lfr.transform(comp_lfr_orig_test)
# comp_lfr_recon_transf_train = comp_lfr.transform(comp_lfr_recon_orig_train)
# comp_lfr_recon_transf_test = comp_lfr.transform(comp_lfr_recon_orig_test)

In [30]:
comp_lfr_X_tr = np.delete(comp_lfr_transf_train.features, comp_prot_index, axis=1)
comp_lfr_X_te = np.delete(comp_lfr_transf_test.features, comp_prot_index, axis=1)
comp_lfr_y_tr = comp_lfr_transf_train.labels.ravel()
comp_lfr_y_te = comp_lfr_transf_test.labels.ravel()

In [95]:
comp_lfr_lr = LogisticRegression(random_state=seed)
comp_lfr_lr.fit(comp_lfr_X_tr, comp_lfr_y_tr)
comp_lfr_lr_pred = comp_lfr_lr.predict(comp_lfr_X_te)
model_accuracy(comp_lfr_y_te, comp_lfr_lr_pred)

ACCURACY: 0.998093422306959
RECALL: 1.0
PRECISION: 0.9953488372093023
F1: 0.9976689976689976


In [32]:
comp_lfr_df = comp_lfr_transf_test.convert_to_dataframe()[0]
comp_lfr_df_priv = comp_lfr_df[comp_lfr_df[comp_protected] == comp_p[0][comp_protected]]
comp_lfr_df_unpriv = comp_lfr_df[comp_lfr_df[comp_protected] == comp_u[0][comp_protected]]

comp_lfr_df_priv_X = comp_lfr_df_priv.drop(labels=[comp_protected, comp_label], axis="columns")
comp_lfr_df_priv_y = comp_lfr_df_priv[comp_label]

comp_lfr_df_unpriv_X = comp_lfr_df_unpriv.drop(labels=[comp_protected, comp_label], axis="columns")
comp_lfr_df_unpriv_y = comp_lfr_df_unpriv[comp_label]

comp_lfr_priv_pred = comp_lfr_lr.predict(comp_lfr_df_priv_X)
comp_lfr_unpriv_pred = comp_lfr_lr.predict(comp_lfr_df_unpriv_X)

comp_lfr_priv_metrics = fairness_metrics(comp_lfr_df_priv_y, comp_lfr_priv_pred)
comp_lfr_unpriv_metrics = fairness_metrics(comp_lfr_df_unpriv_y, comp_lfr_unpriv_pred)

/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [33]:
# optim_options = {
#     "distortion_fun": get_distortion_adult,
#     "epsilon": 0.05,
#     "clist": [0.99, 1.99, 2.99],
#     "dlist": [.1, 0.05, 0]
# }

# op = OptimPreproc(OptTools, optim_options)
# op = op.fit(train)

## Reweighing

### Adult Income

In [34]:
ad_rw_class, ad_rw_recon = ad_scaled.split([0.5])
ad_rw_train, ad_rw_test = ad_rw_class.split([0.66])
ad_rw_recon_train, ad_rw_recon_test = ad_rw_recon.split([0.66])

In [35]:
ad_rw = Reweighing(unprivileged_groups=ad_u,
               privileged_groups=ad_p)
ad_rw.fit(ad_rw_train)

In [36]:
ad_rw_train = ad_rw.transform(ad_rw_train)
ad_rw_test = ad_rw.transform(ad_rw_test)
# ad_rw_recon_train = ad_rw.transform(ad_rw_recon_train)
# ad_rw_recon_test = ad_rw.transform(ad_rw_recon_test)

In [37]:
ad_rw_X_tr = np.delete(ad_rw_train.features, ad_prot_index, axis=1)
ad_rw_y_tr = ad_rw_train.labels.ravel()
ad_rw_X_te = np.delete(ad_rw_test.features, ad_prot_index, axis=1)
ad_rw_y_te = ad_rw_test.labels.ravel()

In [96]:
ad_rw_lr = LogisticRegression(random_state=seed)
ad_rw_lr.fit(ad_rw_X_tr, ad_rw_y_tr)
ad_rw_lr_pred = ad_rw_lr.predict(ad_rw_X_te)
model_accuracy(ad_rw_y_te, ad_rw_lr_pred)

ACCURACY: 0.8034682080924855
RECALL: 0.328125
PRECISION: 0.6852631578947368
F1: 0.4437627811860941


In [39]:
ad_rw_df = ad_rw_test.convert_to_dataframe()[0]
ad_rw_df_priv = ad_rw_df[ad_rw_df[ad_protected] == ad_p[0][ad_protected]]
ad_rw_df_unpriv = ad_rw_df[ad_rw_df[ad_protected] == ad_u[0][ad_protected]]

ad_rw_df_priv_X = ad_rw_df_priv.drop(labels=[ad_protected, ad_label], axis="columns")
ad_rw_df_priv_y = ad_rw_df_priv[ad_label]

ad_rw_df_unpriv_X = ad_rw_df_unpriv.drop(labels=[ad_protected, ad_label], axis="columns")
ad_rw_df_unpriv_y = ad_rw_df_unpriv[ad_label]

ad_rw_priv_pred = ad_rw_lr.predict(ad_rw_df_priv_X)
ad_rw_unpriv_pred = ad_rw_lr.predict(ad_rw_df_unpriv_X)

ad_rw_priv_metrics = fairness_metrics(ad_rw_df_priv_y, ad_rw_priv_pred)
ad_rw_unpriv_metrics = fairness_metrics(ad_rw_df_unpriv_y, ad_rw_unpriv_pred)

/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


### Compas

In [40]:
comp_rw_class, comp_rw_recon = comp_scaled.split([0.5])
comp_rw_train, comp_rw_test = comp_rw_class.split([0.66])
# comp_rw_recon_train, comp_rw_recon_test = comp_rw_recon.split([0.66])

In [41]:
comp_rw = Reweighing(unprivileged_groups=comp_u,
               privileged_groups=comp_p)
comp_rw.fit(comp_rw_train)

In [42]:
comp_rw_train = comp_rw.transform(comp_rw_train)
comp_rw_test = comp_rw.transform(comp_rw_test)
# comp_rw_recon_train = comp_rw.transform(comp_rw_recon_train)
# comp_rw_recon_test = comp_rw.transform(comp_rw_recon_test)

In [43]:
comp_rw_X_tr = np.delete(comp_rw_train.features, comp_prot_index, axis=1)
comp_rw_y_tr = comp_rw_train.labels.ravel()
comp_rw_X_te = np.delete(comp_rw_test.features, comp_prot_index, axis=1)
comp_rw_y_te = comp_rw_test.labels.ravel()

In [ ]:
comp_rw_lr = LogisticRegression(random_state=seed)
comp_rw_lr.fit(comp_rw_X_tr, comp_rw_y_tr)
comp_rw_lr_pred = comp_rw_lr.predict(comp_rw_X_te)
model_accuracy(comp_rw_y_te, comp_rw_lr_pred)

ACCURACY: 0.6310772163965681
RECALL: 0.46975806451612906
PRECISION: 0.6526610644257703
F1: 0.5463071512309496


In [45]:
comp_rw_df = comp_rw_test.convert_to_dataframe()[0]
comp_rw_df_priv = comp_rw_df[comp_rw_df[comp_protected] == comp_p[0][comp_protected]]
comp_rw_df_unpriv = comp_rw_df[comp_rw_df[comp_protected] == comp_u[0][comp_protected]]

comp_rw_df_priv_X = comp_rw_df_priv.drop(labels=[comp_protected, comp_label], axis="columns")
comp_rw_df_priv_y = comp_rw_df_priv[comp_label]

comp_rw_df_unpriv_X = comp_rw_df_unpriv.drop(labels=[comp_protected, comp_label], axis="columns")
comp_rw_df_unpriv_y = comp_rw_df_unpriv[comp_label]

comp_rw_priv_pred = comp_rw_lr.predict(comp_rw_df_priv_X)
comp_rw_unpriv_pred = comp_rw_lr.predict(comp_rw_df_unpriv_X)

comp_rw_priv_metrics = fairness_metrics(comp_rw_df_priv_y, comp_rw_priv_pred)
comp_rw_unpriv_metrics = fairness_metrics(comp_rw_df_unpriv_y, comp_rw_unpriv_pred)

/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


# Analysing Results

## Adult Income

In [46]:
ad_base_binm = BinaryLabelDatasetMetric(dataset=ad_base_test, privileged_groups=ad_p, unprivileged_groups=ad_u)
print_results("ADULT INCOME", ad_base_priv_metrics, ad_base_unpriv_metrics, ad_base_binm)

ADULT INCOME
EQUALIZED ODDS - TPR: 0.056775930219374215
EQUALIZED ODDS - FPR: 0.007699227162234874
STAT PARITY DIFF: -0.19935683391752942
DISPARATE IMPACT: 0.3526738870570151


In [47]:
ad_di_test_repd_pred = ad_dir_test.copy()
ad_di_test_repd_pred.labels = ad_dir_lr.predict(ad_dir_X_te)

ad_di_binm = BinaryLabelDatasetMetric(ad_di_test_repd_pred, privileged_groups=ad_p, unprivileged_groups=ad_u)
print_results("ADULT INCOME", ad_dir_priv_metrics, ad_dir_unpriv_metrics, ad_di_binm)

ADULT INCOME
EQUALIZED ODDS - TPR: -0.03173497792555058
EQUALIZED ODDS - FPR: -0.0077299629800061526
STAT PARITY DIFF: -0.03719305664817925
DISPARATE IMPACT: 0.6737553928655773


In [48]:
ad_lfr_binm = BinaryLabelDatasetMetric(ad_lfr_transf_test, privileged_groups=ad_p, unprivileged_groups=ad_u)
print_results("ADULT INCOME", ad_lfr_priv_metrics, ad_lfr_unpriv_metrics, ad_lfr_binm)

ADULT INCOME
EQUALIZED ODDS - TPR: -0.00026476943706599254
EQUALIZED ODDS - FPR: 0.0022402686993747796
STAT PARITY DIFF: -0.040916358398533414
DISPARATE IMPACT: 0.6292831084087201


In [49]:
ad_rw_binm = BinaryLabelDatasetMetric(ad_rw_test, privileged_groups=ad_p, unprivileged_groups=ad_u)
print_results("REWEIGHING", ad_rw_priv_metrics, ad_rw_unpriv_metrics, ad_rw_binm)

REWEIGHING
EQUALIZED ODDS - TPR: 0.05637547569350487
EQUALIZED ODDS - FPR: 0.0143024615015067
STAT PARITY DIFF: 0.013815352170874512
DISPARATE IMPACT: 1.0586173096772409


## Compas

In [50]:
comp_base_binm = BinaryLabelDatasetMetric(dataset=comp_base_test, privileged_groups=comp_p, unprivileged_groups=comp_u)
print_results("COMPAS", comp_base_priv_metrics, comp_base_unpriv_metrics, comp_base_binm)

COMPAS
EQUALIZED ODDS - TPR: -0.19291596898899283
EQUALIZED ODDS - FPR: -0.0642785188390316
STAT PARITY DIFF: -0.10965885965885963
DISPARATE IMPACT: 0.8199204034512092


In [51]:
comp_di_test_repd_pred = comp_di_test.copy()
comp_di_test_repd_pred.labels = comp_di_lr.predict(comp_di_X_te)

comp_di_binm = BinaryLabelDatasetMetric(comp_di_test_repd_pred, privileged_groups=comp_p, unprivileged_groups=comp_u)
print_results("COMPAS", comp_di_priv_metrics, comp_di_unpriv_metrics, comp_di_binm)

COMPAS
EQUALIZED ODDS - TPR: -0.07881753799633961
EQUALIZED ODDS - FPR: -0.05808571819800079
STAT PARITY DIFF: -0.09365021770682147
DISPARATE IMPACT: 0.8708272859216255


In [52]:
comp_lfr_binm = BinaryLabelDatasetMetric(comp_lfr_transf_test, privileged_groups=comp_p, unprivileged_groups=comp_u)
print_results("COMPAS", comp_lfr_priv_metrics, comp_lfr_unpriv_metrics, comp_lfr_binm)

COMPAS
EQUALIZED ODDS - TPR: 0.0
EQUALIZED ODDS - FPR: 0.008928571428571428
STAT PARITY DIFF: 0.07204577968526465
DISPARATE IMPACT: 1.2001271657924018


In [53]:
comp_rw_binm = BinaryLabelDatasetMetric(comp_rw_test, privileged_groups=comp_p, unprivileged_groups=comp_u)
print_results("COMPAS", comp_rw_priv_metrics, comp_rw_unpriv_metrics, comp_rw_binm)

COMPAS
EQUALIZED ODDS - TPR: -0.14510225192965703
EQUALIZED ODDS - FPR: -0.0899767218951116
STAT PARITY DIFF: -0.035190788957626395
DISPARATE IMPACT: 0.9360151954712247


# Data Reconsruction

In [54]:
k=10

## Adult Income

### Base Data

In [55]:
ad_classifier, ad_recon = ad_scaled.split([0.5])
ad_train, ad_test = ad_classifier.split([0.66])

ad_test_X = np.delete(ad_test.features, ad_prot_index, axis=1)
ad_test_S = ad_test.protected_attributes.ravel()
ad_test_Y = ad_test.labels.ravel()

ad_recon_X = np.delete(ad_recon.features, ad_prot_index, axis=1)
ad_recon_S = ad_recon.protected_attributes.ravel()
ad_recon_Y = ad_recon.labels.ravel()

In [56]:
ad_base_recon_lr = LogisticRegression(random_state=seed, max_iter=500)
ad_base_recon_clf = RandomForestClassifier(random_state=seed)
ad_base_recon_lr.fit(ad_recon_X, ad_recon_S)
ad_base_recon_lr_pred = ad_base_recon_lr.predict(ad_test_X)
print(f"RECONSTRUCTION ACCURACY: {accuracy_score(ad_test_S, ad_base_recon_lr_pred)}")

RECONSTRUCTION ACCURACY: 0.6777456647398844


In [57]:
ad_base_scores_lr = cross_val_score(ad_base_recon_lr, ad_recon_X, ad_recon_S, cv=k)
print(f"MIN: {ad_base_scores_lr.min()}")
print(f" Q1: {np.percentile(ad_base_scores_lr, 25)}")
print(f"MED: {np.percentile(ad_base_scores_lr, 50)}")
print(f" Q3: {np.percentile(ad_base_scores_lr, 75)}")
print(f"MAX: {ad_base_scores_lr.max()}")

MIN: 0.6646191646191646
 Q1: 0.6698402948402948
MED: 0.6805896805896806
 Q3: 0.686015561015561
MAX: 0.6883701883701884


In [58]:
ad_base_scores_clf = cross_val_score(ad_base_recon_clf, ad_recon_X, ad_recon_S, cv=k)
print(f"MIN: {ad_base_scores_clf.min()}")
print(f" Q1: {np.percentile(ad_base_scores_clf, 25)}")
print(f"MED: {np.percentile(ad_base_scores_clf, 50)}")
print(f" Q3: {np.percentile(ad_base_scores_clf, 75)}")
print(f"MAX: {ad_base_scores_clf.max()}")

MIN: 0.6552006552006552
 Q1: 0.6621636288541732
MED: 0.6648239148239148
 Q3: 0.6702497952497952
MAX: 0.6797706797706797


### Disparate Impact Removal

In [59]:
ad_dir_recon_X = ad_dir_recon.features
ad_dir_recon_y = ad_dir_recon.labels.ravel()

In [60]:
ad_dir_recon_lr = LogisticRegression(random_state=seed)
ad_dir_recon_clf = RandomForestClassifier(random_state=seed)
ad_dir_recon_lr_scores = fairness_cv(ad_dir_recon_lr, ad_dir, ad_dir_recon_X, ad_dir_recon_y, ad_recon.feature_names, ad_protected, ad_recon.label_names[0], prot_index=ad_prot_index, cv=k)
ad_dir_recon_clf_scores = fairness_cv(ad_dir_recon_clf, ad_dir, ad_dir_recon_X, ad_dir_recon_y, ad_recon.feature_names, ad_protected, ad_recon.label_names[0], prot_index=ad_prot_index, cv=k)

/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-pack

In [61]:
print(f"MIN: {ad_dir_recon_lr_scores.min()}")
print(f" Q1: {np.percentile(ad_dir_recon_lr_scores, 25)}")
print(f"MED: {np.percentile(ad_dir_recon_lr_scores, 50)}")
print(f" Q3: {np.percentile(ad_dir_recon_lr_scores, 75)}")
print(f"MAX: {ad_dir_recon_lr_scores.max()}")

MIN: 0.5687960687960688
 Q1: 0.6394348894348895
MED: 0.6482391482391483
 Q3: 0.6535626535626536
MAX: 0.6666666666666666


In [62]:
print(f"MIN: {ad_dir_recon_clf_scores.min()}")
print(f" Q1: {np.percentile(ad_dir_recon_clf_scores, 25)}")
print(f"MED: {np.percentile(ad_dir_recon_clf_scores, 50)}")
print(f" Q3: {np.percentile(ad_dir_recon_clf_scores, 75)}")
print(f"MAX: {ad_dir_recon_clf_scores.max()}")

MIN: 0.3927108927108927
 Q1: 0.42342342342342343
MED: 0.43652743652743653
 Q3: 0.45382882882882886
MAX: 0.481981981981982


### Learning Fair Representations

In [63]:
ad_lfr_recon_X = ad_lfr_recon.features
ad_lfr_recon_y = ad_lfr_recon.labels.ravel()

In [64]:
ad_lfr_recon_lr = LogisticRegression(random_state=seed)
ad_lfr_recon_clf = RandomForestClassifier(random_state=seed)
ad_lfr_lr_scores = fairness_cv(ad_lfr_recon_lr, ad_lfr, ad_lfr_recon_X, ad_lfr_recon_y, ad_lfr_recon.feature_names, ad_protected, ad_lfr_recon.label_names[0], ad_prot_index, cv=k)
ad_lfr_clf_scores = fairness_cv(ad_lfr_recon_clf, ad_lfr, ad_lfr_recon_X, ad_lfr_recon_y, ad_lfr_recon.feature_names, ad_protected, ad_lfr_recon.label_names[0], ad_prot_index, cv=k)

/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-pack

In [65]:
print(f"MIN: {ad_lfr_lr_scores.min()}")
print(f" Q1: {np.percentile(ad_lfr_lr_scores, 25)}")
print(f"MED: {np.percentile(ad_lfr_lr_scores, 50)}")
print(f" Q3: {np.percentile(ad_lfr_lr_scores, 75)}")
print(f"MAX: {ad_lfr_lr_scores.max()}")

MIN: 0.39926289926289926
 Q1: 0.42065929565929566
MED: 0.42588042588042585
 Q3: 0.4334561834561834
MAX: 0.4451269451269451


In [66]:
print(f"MIN: {ad_lfr_clf_scores.min()}")
print(f" Q1: {np.percentile(ad_lfr_clf_scores, 25)}")
print(f"MED: {np.percentile(ad_lfr_clf_scores, 50)}")
print(f" Q3: {np.percentile(ad_lfr_clf_scores, 75)}")
print(f"MAX: {ad_lfr_clf_scores.max()}")

MIN: 0.21171171171171171
 Q1: 0.2508190008190008
MED: 0.26453726453726456
 Q3: 0.27774365274365276
MAX: 0.3050778050778051


### Reweighing

In [67]:
ad_rw_recon_X = ad_rw_recon.features
ad_rw_recon_y = ad_rw_recon.labels.ravel()

In [68]:
ad_rw_recon_lr = LogisticRegression(random_state=seed)
ad_rw_recon_clf = RandomForestClassifier(random_state=seed)
ad_rw_lr_scores = fairness_cv(ad_rw_recon_lr, ad_rw, ad_rw_recon_X, ad_rw_recon_y, ad_rw_recon.feature_names, ad_protected, ad_rw_recon.label_names[0], ad_prot_index, cv=k)
ad_rw_clf_scores = fairness_cv(ad_rw_recon_clf, ad_rw, ad_rw_recon_X, ad_rw_recon_y, ad_rw_recon.feature_names, ad_protected, ad_rw_recon.label_names[0], ad_prot_index, cv=k)

/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-pack

In [69]:
print(f"MIN: {ad_rw_lr_scores.min()}")
print(f" Q1: {np.percentile(ad_rw_lr_scores, 25)}")
print(f"MED: {np.percentile(ad_rw_lr_scores, 50)}")
print(f" Q3: {np.percentile(ad_rw_lr_scores, 75)}")
print(f"MAX: {ad_rw_lr_scores.max()}")

MIN: 0.6621621621621622
 Q1: 0.6735257985257985
MED: 0.6816134316134317
 Q3: 0.6832514332514332
MAX: 0.6855036855036855


In [70]:
print(f"MIN: {ad_rw_clf_scores.min()}")
print(f" Q1: {np.percentile(ad_rw_clf_scores, 25)}")
print(f"MED: {np.percentile(ad_rw_clf_scores, 50)}")
print(f" Q3: {np.percentile(ad_rw_clf_scores, 75)}")
print(f"MAX: {ad_rw_clf_scores.max()}")

MIN: 0.6515151515151515
 Q1: 0.6629811629811629
MED: 0.6687141687141687
 Q3: 0.6715806715806716
MAX: 0.6826371826371826


## Compas

### No FC

In [71]:
comp_classifier, comp_recon = comp_scaled.split([0.5])
comp_train, comp_test = comp_classifier.split([0.66])

comp_test_X = np.delete(comp_test.features, comp_prot_index, axis=1)
comp_test_S = comp_test.protected_attributes.ravel()
comp_test_Y = comp_test.labels.ravel()

comp_recon_X = np.delete(comp_recon.features, comp_prot_index, axis=1)
comp_recon_S = comp_recon.protected_attributes.ravel()
comp_recon_Y = comp_recon.labels.ravel()

In [72]:
comp_base_recon_lr = LogisticRegression(random_state=seed, max_iter=500)
comp_base_recon_clf = RandomForestClassifier(random_state=seed)
comp_base_recon_lr.fit(comp_recon_X, comp_recon_S)
comp_base_recon_lr_pred = comp_base_recon_lr.predict(comp_test_X)
print(f"RECONSTRUCTION ACCURACY: {accuracy_score(comp_test_S, comp_base_recon_lr_pred)}")

RECONSTRUCTION ACCURACY: 0.6711153479504289


In [73]:
comp_base_lr_scores = cross_val_score(comp_base_recon_lr, comp_recon_X, comp_recon_S, cv=k)
print(f"MIN: {comp_base_lr_scores.min()}")
print(f" Q1: {np.percentile(comp_base_lr_scores, 25)}")
print(f"MED: {np.percentile(comp_base_lr_scores, 50)}")
print(f" Q3: {np.percentile(comp_base_lr_scores, 75)}")
print(f"MAX: {comp_base_lr_scores.max()}")

MIN: 0.6245954692556634
 Q1: 0.6642394822006472
MED: 0.6753246753246753
 Q3: 0.6882565250283696
MAX: 0.7077922077922078


In [74]:
comp_base_clf_scores = cross_val_score(comp_base_recon_clf, comp_recon_X, comp_recon_S, cv=k)
print(f"MIN: {comp_base_clf_scores.min()}")
print(f" Q1: {np.percentile(comp_base_clf_scores, 25)}")
print(f"MED: {np.percentile(comp_base_clf_scores, 50)}")
print(f" Q3: {np.percentile(comp_base_clf_scores, 75)}")
print(f"MAX: {comp_base_clf_scores.max()}")

MIN: 0.6136363636363636
 Q1: 0.626917580801076
MED: 0.64829466649855
 Q3: 0.6607142857142857
MAX: 0.685064935064935


### Disparate Impact Remover

In [75]:
comp_dir_recon_X = comp_di_recon.features
comp_dir_recon_y = comp_di_recon.labels.ravel()

In [76]:
comp_di_recon_lr = LogisticRegression(random_state=seed)
comp_di_recon_clf = RandomForestClassifier(random_state=seed)
comp_di_lr_scores = fairness_cv(comp_di_recon_lr, comp_di, comp_dir_recon_X, comp_dir_recon_y, comp_di_recon.feature_names, comp_protected, comp_di_recon.label_names[0], comp_prot_index, cv=k)
comp_di_clf_scores = fairness_cv(comp_di_recon_clf, comp_di, comp_dir_recon_X, comp_dir_recon_y, comp_di_recon.feature_names, comp_protected, comp_di_recon.label_names[0], comp_prot_index, cv=k)

/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-pack

In [77]:
print(f"MIN: {comp_di_lr_scores.min()}")
print(f" Q1: {np.percentile(comp_di_lr_scores, 25)}")
print(f"MED: {np.percentile(comp_di_lr_scores, 50)}")
print(f" Q3: {np.percentile(comp_di_lr_scores, 75)}")
print(f"MAX: {comp_di_lr_scores.max()}")

MIN: 0.5974025974025974
 Q1: 0.6501623376623377
MED: 0.6623376623376623
 Q3: 0.6737012987012987
MAX: 0.6948051948051948


In [78]:
print(f"MIN: {comp_di_clf_scores.min()}")
print(f" Q1: {np.percentile(comp_di_clf_scores, 25)}")
print(f"MED: {np.percentile(comp_di_clf_scores, 50)}")
print(f" Q3: {np.percentile(comp_di_clf_scores, 75)}")
print(f"MAX: {comp_di_clf_scores.max()}")

MIN: 0.6136363636363636
 Q1: 0.6347402597402597
MED: 0.6493506493506493
 Q3: 0.6623376623376623
MAX: 0.6785714285714286


### LFR

In [79]:
comp_lfr_recon_X = comp_lfr_recon.features
comp_lfr_recon_y = comp_lfr_recon.labels.ravel()

In [80]:
comp_lfr_recon_lr = LogisticRegression(random_state=seed)
comp_lfr_recon_clf = RandomForestClassifier(random_state=seed)
comp_lfr_lr_scores = fairness_cv(comp_lfr_recon_lr, comp_lfr, comp_lfr_recon_X, comp_lfr_recon_y, comp_lfr_recon.feature_names, comp_protected, comp_lfr_recon.label_names[0], comp_prot_index, cv=k)
comp_lfr_clf_scores = fairness_cv(comp_lfr_recon_clf, comp_lfr, comp_lfr_recon_X, comp_lfr_recon_y, comp_lfr_recon.feature_names, comp_protected, comp_lfr_recon.label_names[0], comp_prot_index, cv=k)

/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-pack

In [81]:
print(f"MIN: {comp_lfr_lr_scores.min()}")
print(f" Q1: {np.percentile(comp_lfr_lr_scores, 25)}")
print(f"MED: {np.percentile(comp_lfr_lr_scores, 50)}")
print(f" Q3: {np.percentile(comp_lfr_lr_scores, 75)}")
print(f"MAX: {comp_lfr_lr_scores.max()}")

MIN: 0.6233766233766234
 Q1: 0.637987012987013
MED: 0.6542207792207793
 Q3: 0.6696428571428571
MAX: 0.7142857142857143


In [82]:
print(f"MIN: {comp_lfr_clf_scores.min()}")
print(f" Q1: {np.percentile(comp_lfr_clf_scores, 25)}")
print(f"MED: {np.percentile(comp_lfr_clf_scores, 50)}")
print(f" Q3: {np.percentile(comp_lfr_clf_scores, 75)}")
print(f"MAX: {comp_lfr_clf_scores.max()}")

MIN: 0.6201298701298701
 Q1: 0.6363636363636364
MED: 0.6623376623376623
 Q3: 0.672077922077922
MAX: 0.7142857142857143


### Reweighing

In [83]:
comp_rw_recon_X = comp_rw_recon.features
comp_rw_recon_y = comp_rw_recon.labels.ravel()

In [84]:
comp_rw_recon_lr = LogisticRegression(random_state=seed)
comp_rw_recon_clf = RandomForestClassifier(random_state=seed)
comp_rw_lr_scores = fairness_cv(comp_rw_recon_lr, comp_rw, comp_rw_recon_X, comp_rw_recon_y, comp_rw_recon.feature_names, comp_protected, comp_rw_recon.label_names[0], comp_prot_index, cv=k)
comp_rw_clf_scores = fairness_cv(comp_rw_recon_clf, comp_rw, comp_rw_recon_X, comp_rw_recon_y, comp_rw_recon.feature_names, comp_protected, comp_rw_recon.label_names[0], comp_prot_index, cv=k)

/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/nickday/.local/lib/python3.10/site-pack

In [85]:
print(f"MIN: {comp_rw_lr_scores.min()}")
print(f" Q1: {np.percentile(comp_rw_lr_scores, 25)}")
print(f"MED: {np.percentile(comp_rw_lr_scores, 50)}")
print(f" Q3: {np.percentile(comp_rw_lr_scores, 75)}")
print(f"MAX: {comp_rw_lr_scores.max()}")

MIN: 0.6428571428571429
 Q1: 0.6599025974025974
MED: 0.6753246753246753
 Q3: 0.689935064935065
MAX: 0.7305194805194806


In [86]:
print(f"MIN: {comp_rw_clf_scores.min()}")
print(f" Q1: {np.percentile(comp_rw_clf_scores, 25)}")
print(f"MED: {np.percentile(comp_rw_clf_scores, 50)}")
print(f" Q3: {np.percentile(comp_rw_clf_scores, 75)}")
print(f"MAX: {comp_rw_clf_scores.max()}")

MIN: 0.6233766233766234
 Q1: 0.6363636363636364
MED: 0.6525974025974026
 Q3: 0.6672077922077921
MAX: 0.6915584415584416
